In [1]:
import pandas as pd


from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [2]:

def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [3]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed

In [4]:

# EXTRACT IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.corte_excel import extract_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.sga_335 import extract_sga_335

In [5]:
# TRANSFORM IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_335 import ( 
    preprocess_335
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_380 import ( 
    preprocess_380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.cuismp_sharepoint import ( 
    preprocess_df_cid_cuismp_sharepoint
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.corte_excel import ( 
    preprocess_corte_excel
)

In [6]:
# MERGE IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.excel_sga.excel_sga import ( 
merge_sga_335_corte_excel_sharepoint_cuismp_sga380
)

In [7]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.o11_indisponibilidad_validator import validate_indisponibilidad_v2
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.o11_indisponibilidad_validator import build_failure_messages_indisponibilidad




In [8]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE FINAL 4_20250722_170734.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "335_20250724_200636.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB (2)_20250719_215633.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "380_20250724_200636.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS_20250724_200636.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS_20250724_200636.docx"


In [9]:
df_corte_excel = extract_corte_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=0)
df_sga_dinamico_335 =  extract_sga_335(SAVE_DIR_EXTRACT_SGA_335)
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)


In [10]:
df_corte_excel = preprocess_corte_excel(df_corte_excel)
df_cid_cuismp_sharepoint = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
df_sga_dinamico_335 = preprocess_335(df_sga_dinamico_335)
df_sga_dinamico_380 = preprocess_380(df_sga_dinamico_380)

In [11]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )

In [12]:
df_vali = validate_indisponibilidad_v2(df_matched_corte_sga335_Sharepoint_cuismp_sga380)
row = df_vali[df_vali['nro_incidencia'] == '21853199']
row


,nro_incidencia,FECHA Y HORA INICIO,FECHA Y HORA FIN,CUISMP_sga_dinamico_335_excel_matched,TIPO CASO,AVERÍA,TIEMPO (HH:MM),COMPONENTE,DF,OBSERVACIÓN,CID,FIN-INICIO (HH:MM),DETERMINACIÓN DE LA CAUSA,RESPONSABILIDAD,TIPO REPORTE,Duracion entero,Agrupación entero,CODINCIDENCEPADRE,MASIVO,MEDIDAS CORRECTIVAS Y/O PREVENTIVAS TOMADAS,TIPO DE INCIDENCIA,TIEMPO INTERRUPCION,INDISPONIBILIDAD,num_A_traves,TIEMPO (HH:MM)_trimed,FIN-INICIO (HH:MM)_trimed,FECHA_Y_HORA_INICIO_fmt,FECHA_Y_HORA_FIN_fmt,duration_diff_corte_sec,diff_corte_sec_hhmm,fin_inicio_hhmm_column_corte_to_minutes,duration_diff_corte_min,extracted_hour,canal_ingreso,interrupcion_inicio,fecha_generacion,interrupcion_fin,cid,tipo_caso,tipificacion_problema,it_determinacion_de_la_causa,it_medidas_tomadas,it_conclusiones,tiempo_interrupcion,tipificacion_interrupcion,tipificacion_tipo,fecha_comunicacion_cliente,masivo,codincidencepadre,fecha_generacion_truncated,interrupcion_inicio_truncated,interrupcion_fin_truncated,Expected_Inicio_truncated,Expected_Inicio_truncated_fm,interrupcion_fin_truncated_fm,duration_diff_335,duration_diff_335_sec,diff_335_sec_hhmm,duration_diff_335_min,_merge,CUISMP_sharepoint_cid_cuismp,Distrito Fiscal,%Disponibilidad,BW contratado,SEDE,CID NUEVO,Unnamed: 7,Unnamed: 8,Unnamed: 9,sum_paradas,clock_stops,clock_stops_paragraph,clock_stops_paragraph_header,clock_stops_paragraph_periodos,clock_stops_paragraph_footer,indisponibilidad_header_v2,indisponibilidad_periodos_v2,indisponibilidad_total_v2,indisponibilidad_header_match_v2,indisponibilidad_periodos_match_v2,indisponibilidad_total_match_v2,Validation_OK_v2,fail_count_v2
27,21853199,2025-07-15 17:49:00,2025-07-16 02:07:00,2020,SIN SERVICIO-MONITOREO,FALLA DE ENERGIA (CLIENTE) - SERVICIO RESTABLECIDO POR POWER ON,00:01,COMPONENTE II,Ancash,Se generó ticket para la revisión del servicio de datos de la sede Huari1,21096611,08:18,"El inconveniente se originó por un problema de energía en el local del cliente, afectando los equipos de comunicaciones.",CLIENTE,PROACTIVO,0,Menor a 1h,21853199,No,"A través de los Sistemas de Monitoreo de Claro, de manera proactiva se identificó la pérdida de gestión del servicio de datos del cliente identificado con el CUISMP 2020. Se generó un ticket el día 15/07/2025 a las 17:49 horas. Inmediatamente, Claro revisó el servicio, encontrando pérdida de conectividad con el equipo ubicado en la sede del cliente. Ante ello, se gestionó el desplazamiento de personal técnico especializado a nuestro punto de presencia para las revisiones correspondientes del enlace, donde se descartó eventos en el tramo de fibra óptica. Finalmente, el servicio del cliente se restableció por encendido de equipos el día 16/07/2025 a las 02:07 horas.",REPORTE PREVIO - Calidad,"0,01",Se tuvo indisponibilidad por parte del cliente para continuar los trabajos el/los día(s) 15/07/2025 17:50:00 hasta el día 16/07/2025 02:07:00(Total de horas sin acceso a la sede: 8:17 horas),1,00:01,08:18,15/07/2025 17:49,16/07/2025 02:07,0 days 08:18:00,08:18,498.00,498,0,Proactivo,2025-07-15 17:49:03,2025-07-15 17:49:03,2025-07-16 02:07:00,21096611,SIN SERVICIO-MONITOREO,FALLA DE ENERGIA (CLIENTE) - SERVICIO RESTABLECIDO POR POWER ON,"COMPONENTE II - El inconveniente se originó por un problema de energía en el local del cliente, afectando los equipos de comunicaciones.","A través de los Sistemas de Monitoreo de Claro, de manera proactiva se identificó la pérdida de gestión del servicio de datos del cliente identificado con el CUISMP 2020. Se generó un ticket el día 15/07/2025 a las 17:49 horas. Inmediatamente, Claro revisó el servicio, encontrando pérdida de conectividad con el equipo ubicado en la sede del cliente. Ante ello, se gestionó el desplazamiento de personal técnico especializado a nuestro punto de presencia para las revisiones correspondientes del enlace, donde se descartó eventos en el tramo de fibra óptica. Finalmente, el servicio del cliente se restableció por encendido de equipos el día 16/07/2025 a las 0

In [13]:
df_mess = build_failure_messages_indisponibilidad(df_vali)
df_mess

,nro_incidencia,mensaje,TIPO REPORTE,objetivo
